In [1]:
# Script utilizado para criar modelo de estrutura de dados utilizada
# para o modelo de algoritmo adaptativo

import os 
import pandas as pd
import re
import pickle

path = r'C:\Users\leofb\Desktop\Work\Python\Resultados'

# Dicionário contendo todos os dados dos resultados

data = {}

# Dicionário contendo cada dataset

dataset = {}

# Lista contendo cada pasta referente a cada modulação

folders = []

for folder in os.listdir(path):
        folders.append(folder)
        
for folder in folders:
    data.update({folder: {}})
    dataset.update({folder: []})
    for file in os.listdir(path + '\\' + folder):
        if file.endswith('.txt'):
#             dataset[folder].append(pd.read_csv(path + '\\' + folder + '\\' + file, index_col=None, delimiter=',', 
#                   names=["SNR", "MOS", "BER"]))
            df = pd.read_csv(path + '\\' + folder + '\\' + file, index_col=None, delimiter=',', 
                  names=["SNR", "MOS", "BER"])
    
# Adiciona coluna com tipo de codificação de 
# canal em cada data set   

            idx_len = len(df.index)
            code = re.search("\(.*?\)", file)
            if code != None:
                code = code.group(0)
                
# Obtem o valor de n e k dos códigos RS e o overhead

                n = re.split("[^0-9]+", code)[1]
                k = re.split("[^0-9]+", code)[2]
                rate = [int(n) / int(k)] * idx_len
            
                code = ['RS' + code.replace('-', ', ')] * idx_len
                
            else:
                rate = [1.0] * idx_len
                code = ['No Code'] * idx_len
            df['FEC'] = code
            df['RATE'] = rate   
            dataset[folder].append(df)

for modulation, tables in dataset.items():
    for table in tables:
        for index, row in table.iterrows():
            tup = (row['MOS'], row['BER'], row['FEC'], row['RATE'])
            data[modulation].setdefault(row['SNR'], [])
            data[modulation][row['SNR']].append(tup)

# Salvar dicionario de dados 

a_file = open("data.pkl", "wb")
pickle.dump(data, a_file)
a_file.close()